# Step 1: Imports


In [2]:
import pandas as pd


### Step 2: Reading in Data

In [3]:
data = pd.read_csv(r'C:\Users\Owner\Documents\Projects\application\SM64 Runs\data_120 Star.csv')

In [4]:
data.head()


,Unnamed: 0,id,place,speedrun_link,submitted_date,primary_time_seconds,real_time_seconds,player_id,player_name,player_country,platform,verified
0,0,zx81j5gm,1,https://www.speedrun.com/sm64/run/zx81j5gm,2022-11-18T22:01:40Z,5855,5855,kjprmwk8,Weegee,United States,N64,Yes
1,1,m3ko474y,2,https://www.speedrun.com/sm64/run/m3ko474y,2023-09-23T18:07:19Z,5859,5859,j2ylqn68,Karin,Japan,N64,Yes
2,2,ydjpo90y,3,https://www.speedrun.com/sm64/run/ydjpo90y,2023-08-18T15:00:51Z,5862,5862,kj9yz0wx,Liam,United States,N64,No
3,3,m7390n5y,3,https://www.speedrun.com/sm64/run/m7390n5y,2023-09-15T16:08:40Z,5862,5862,jo2mr9n8,marlene,NaN,N64,Yes
4,4,zgd6pl0z,5,https://www.speedrun.com/sm64/run/zgd6pl0z,2022-03-01T19:41:14Z,5870,5870,v814mkp8,cheese,Spain,N64,Yes


Need to drop the first column

In [18]:
# Creating a new df object set to a mask of the original dataframe.
# Dropping the two id columns and the speedrun link
df = data[['place', 'submitted_date',
        'primary_time_seconds', 'real_time_seconds', 'player_name',
        'player_country', 'platform', 'verified']]
df


,place,submitted_date,primary_time_seconds,real_time_seconds,player_name,player_country,platform,verified
0,1,2022-11-18T22:01:40Z,5855,5855,Weegee,United States,N64,Yes
1,2,2023-09-23T18:07:19Z,5859,5859,Karin,Japan,N64,Yes
2,3,2023-08-18T15:00:51Z,5862,5862,Liam,United States,N64,No
3,3,2023-09-15T16:08:40Z,5862,5862,marlene,NaN,N64,Yes
4,5,2022-03-01T19:41:14Z,5870,5870,cheese,Spain,N64,Yes
...,...,...,...,...,...,...,...,...
495,496,2022-08-09T23:11:52Z,7473,7473,daiden,Canada,N64,Yes
496,497,2023-07-31T19:52:03Z,7474,7474,disa,France,EMU,Yes
497,498,2022-07-31T06:39:39Z,7475,7475,Malkierian,United States,EMU,Yes
498,499,2022-04-28T23:58:29Z,7477,7477,GeoGringo,United States,N64,Yes


Next I want to check if there is any difference in the primary time and real time columns
Both are exactly the same, Im gonna comment out and rewrite the code block into a function.


In [104]:
# c=1
# for pts, rts in zip(df['primary_time_seconds'], df['real_time_seconds']):
#     if pts != rts:
#         print(c)
#     c += 1

def time_checker(df):
    """
    This function is to check if there is a difference between the two time 
    columns. This is to check before dropping the primary time column. It runs 
    a list comprehension and to identify the placement, it uses the index plus
    one. This is because placements start at 1st place.
    
    If any two column cells don't match, they will be placed into the list and
    individually printed out into the console with their respective index.
    """
    place_list = [
        placement + 1 for placement, (pts, rts) in 
        enumerate(zip(df['primary_time_seconds'], df['real_time_seconds'])) 
        if pts != rts
    ]
    # Returns either the list of non-matches or the cleaned df.
    if place_list:
        for placement in place_list:
            print(
                f"""At placement {placement}, index {placement-1}, the primary 
                and real time seconds are different."""
                )
        return place_list
    else:
        return df.drop(columns="primary_time_seconds", axis=1)
ndf = time_checker(df)

Afterwards, I want to check for nulls.

In [62]:
ndf.info()
# I see now that with the there are around 50 nulls in the names, country, and
# submitted date column. This will be annoying when reading into a dashboard,
# but I can deal with it.

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   place              500 non-null    int64 
 1   submitted_date     449 non-null    object
 2   real_time_seconds  500 non-null    int64 
 3   player_name        460 non-null    object
 4   player_country     445 non-null    object
 5   platform           500 non-null    object
 6   verified           500 non-null    object
dtypes: int64(2), object(5)
memory usage: 27.5+ KB


In [105]:
# This corrects the date by removing the seconds and changing it to m/d/y.
# date_list = []
# for date in df['submitted_date']:
#     if pd.isna(date) == False:
#         c_date = date[5:10].replace("-", "/") + "/" + date[:4]
#         date_list.append(c_date)
#     else:
#         date_list.append(date)
# date_list

def date_fixer(test_df):
    """
    Using list comprehension, this function pulls the date and arranges into
    m/d/y format for better readability while also removing the seconds from
    the submission. If the value is null, its added to the list as is.
    The list replaces the input dataframe and then returned.
    """
    test_df.submitted_date = [
        date[5:10].replace("-", "/") + "/" + date[:4]
        if pd.isna(date) == False else date for date in test_df['submitted_date']
    ]
    return test_df

bndf =  date_fixer(ndf)
bndf

,place,submitted_date,real_time_seconds,player_name,player_country,platform,verified
0,1,11/18/2022,5855,Weegee,United States,N64,Yes
1,2,09/23/2023,5859,Karin,Japan,N64,Yes
2,3,08/18/2023,5862,Liam,United States,N64,No
3,3,09/15/2023,5862,marlene,NaN,N64,Yes
4,5,03/01/2022,5870,cheese,Spain,N64,Yes
...,...,...,...,...,...,...,...
495,496,08/09/2022,7473,daiden,Canada,N64,Yes
496,497,07/31/2023,7474,disa,France,EMU,Yes
497,498,07/31/2022,7475,Malkierian,United States,EMU,Yes
498,499,04/28/2022,7477,GeoGringo,United States,N64,Yes


Now, I want to fill the null values as "Non-specified".

In [111]:
bndf.fillna('Non-Specified')

,place,submitted_date,real_time_seconds,player_name,player_country,platform,verified
0,1,11/18/2022,5855,Weegee,United States,N64,Yes
1,2,09/23/2023,5859,Karin,Japan,N64,Yes
2,3,08/18/2023,5862,Liam,United States,N64,No
3,3,09/15/2023,5862,marlene,Non-Specified,N64,Yes
4,5,03/01/2022,5870,cheese,Spain,N64,Yes
...,...,...,...,...,...,...,...
495,496,08/09/2022,7473,daiden,Canada,N64,Yes
496,497,07/31/2023,7474,disa,France,EMU,Yes
497,498,07/31/2022,7475,Malkierian,United States,EMU,Yes
498,499,04/28/2022,7477,GeoGringo,United States,N64,Yes


I believe on the cleaning, Im done. Next is throwing it the code into a pipeline to streamline this.